In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

## datetime

In [ ]:
# object -> 날짜 형식으로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['minute'] = train['datetime'].dt.minute
train['second'] = train['datetime'].dt.second
# 요일 (0: 월 ~ 6: 일)
train['dayofweek'] = train['datetime'].dt.dayofweek

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['minute'] = test['datetime'].dt.minute
test['second'] = test['datetime'].dt.second
# 요일 (0: 월 ~ 6: 일)
test['dayofweek'] = test['datetime'].dt.dayofweek

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['minute'].value_counts()

In [ ]:
train['second'].value_counts()

## 이상치 (outlier), 오류 데이터 

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(train['windspeed'])

In [ ]:
train['windspeed'].value_counts()

In [ ]:
# 조건
cond = (train['windspeed'] == 0)
train.loc[cond, 'windspeed'] = np.nan

In [ ]:
train['windspeed'] = train['windspeed'].interpolate().bfill()

In [ ]:
train['windspeed'].value_counts()

## temp, atemp

In [ ]:
train.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 12))
sns.heatmap(train.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
import numpy as np

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot((train['count']))

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(np.log1p(train['count']))

In [ ]:
a = 10
b = np.log(a)
print(f'로그 변환된 값: {b}')

print(f'역변환: {np.exp(b)}')

In [ ]:
a = 10
b = np.log1p(a)
print(f'로그 변환된 값: {b}')

print(f'역변환: {np.expm1(b)}')

## day

In [ ]:
train['day'].value_counts().sort_index()

In [ ]:
test['day'].value_counts().sort_index()

## 학습용 데이터셋

In [ ]:
train.columns

In [ ]:
# 입력 데이터로 활용한 독립변수(x)를 정의합니다.
features = [
#     'datetime', 
    'season', 
    'holiday', 
    'workingday', 
    'weather', 
    'temp',
#     'atemp', 
    'humidity', 
    'windspeed', 
    'year', # 연도 추가
    'month', # 월 추가
#     'day', # 일자 추가
    'hour', # 시간 추가
    'dayofweek', # 요일 추가
]

In [ ]:
# AI 모델이 예측할 자전거의 대여량 (Y)를 정의합니다.
target = 'count'

In [ ]:
train[features].head(3)

In [ ]:
test.isnull().sum()

In [ ]:
train.info()

## x, y 정의

In [ ]:
x = train[features]
x.head()

In [ ]:
y = train[target]
# 로그 변환
y = np.log1p(y)
y.head()

## 모델 생성 및 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor

SEED = 123

In [ ]:
# STEP 1) 모델 생성 (RandomForest)
model = RandomForestRegressor(random_state=SEED)

In [ ]:
# STEP 2) 모델 학습 (데이터 기반 학습) -> x, y 
model.fit(x, y)

In [ ]:
# STEP 3) 예측 prediction -> test 데이터프레임 예측
prediction = model.predict(test[features])
prediction = np.expm1(prediction)

In [ ]:
prediction[:10]

## submission

In [ ]:
# 정답지에 결과 입력
submission['count'] = prediction

In [ ]:
submission

## 정답 제출

In [ ]:
# submission을 파일(csv)로 저장 -> 우측메뉴에서 - Data - Output - 파일 잘 저장되었는지 확인
submission.to_csv('submission-04.csv', index=False)

In [ ]:
# 첫 번째 제출
# Score: 1.36902

# 두 번째 제출 (datetime 시간 관련된 특성 값을 추가: 연도, 월, 일, 시, 요일)
# Score: 0.47745

# 3번째 제출 (atemp 제거 (다중공성선: 상관관계 계수가 0.98로 상당히 높음), day컬럼 제거: train/test 분포가 다름)
# Score: 0.47007